<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Шаг-1.-Загрузка-данных." data-toc-modified-id="Шаг-1.-Загрузка-данных.-1">Шаг 1. Загрузка данных.</a></span></li><li><span><a href="#Шаг-2.--Проверка-гипотезы-о-неизменности-качества-модели-после-трансформации-признаков." data-toc-modified-id="Шаг-2.--Проверка-гипотезы-о-неизменности-качества-модели-после-трансформации-признаков.-2">Шаг 2.  Проверка гипотезы о неизменности качества модели после трансформации признаков.</a></span></li><li><span><a href="#Шаг-3.-Алгоритм-преобразования-матрицы-признаков." data-toc-modified-id="Шаг-3.-Алгоритм-преобразования-матрицы-признаков.-3">Шаг 3. Алгоритм преобразования матрицы признаков.</a></span></li><li><span><a href="#Шаг-4.-Проверка-алгоритма-преобразования-матрицы-признаков." data-toc-modified-id="Шаг-4.-Проверка-алгоритма-преобразования-матрицы-признаков.-4">Шаг 4. Проверка алгоритма преобразования матрицы признаков.</a></span><ul class="toc-item"><li><span><a href="#Шаг-4.1.-Подготовительный-этап-реализации-алгоритма-преобразования." data-toc-modified-id="Шаг-4.1.-Подготовительный-этап-реализации-алгоритма-преобразования.-4.1">Шаг 4.1. Подготовительный этап реализации алгоритма преобразования.</a></span></li><li><span><a href="#Шаг-4.2.-Проверка-модели-на-признаках-до-трансформации." data-toc-modified-id="Шаг-4.2.-Проверка-модели-на-признаках-до-трансформации.-4.2">Шаг 4.2. Проверка модели на признаках до трансформации.</a></span></li><li><span><a href="#Шаг-4.3.-Проверка-модели-на-признаках-после-трансформации." data-toc-modified-id="Шаг-4.3.-Проверка-модели-на-признаках-после-трансформации.-4.3">Шаг 4.3. Проверка модели на признаках после трансформации.</a></span></li><li><span><a href="#Шаг-4.4.-Дешифрование-преобразованной-матрицы-признаков." data-toc-modified-id="Шаг-4.4.-Дешифрование-преобразованной-матрицы-признаков.-4.4">Шаг 4.4. Дешифрование преобразованной матрицы признаков.</a></span></li></ul></li><li><span><a href="#Шаг-5.-Итоги-исследования." data-toc-modified-id="Шаг-5.-Итоги-исследования.-5">Шаг 5. Итоги исследования.</a></span><ul class="toc-item"><li><span><a href="#Шаг-5.1.-Описание-хода-исследования." data-toc-modified-id="Шаг-5.1.-Описание-хода-исследования.-5.1">Шаг 5.1. Описание хода исследования.</a></span></li><li><span><a href="#Шаг-5.2.-Общий-вывод." data-toc-modified-id="Шаг-5.2.-Общий-вывод.-5.2">Шаг 5.2. Общий вывод.</a></span></li></ul></li></ul></div>

# Защита персональных данных клиентов

**Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.**

**Необходимо защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.**

## Шаг 1. Загрузка данных.

In [1]:
# импорт основных библиотек
import pandas as pd
import numpy as np

# импорт библиотек для деления выборок, масштабирования признаков и pipline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# импорт модели и метрик
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
# чтение и запись в переменную данных датасета с применением конструкции try-except
try:
    df = pd.read_csv('/Users/regki/Downloads/insurance.csv')
except:
    df = pd.read_csv('/datasets/insurance.csv')

In [3]:
# вывод на экран таблицы
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [4]:
# обзор данных таблицы
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


 - В датасете 5000 строк с данными, 5 столбцов, 3 из которых имеют данные в целочисленном формате(пол, возраст и количество страховых выплат у клиента), 2 в вещественном(зарплата и возраст). 
 - Пропуски в данных отсутствуют. 
 - Зарплата и возраст имеют один знак в дробной части, выраженный числом 0, поэтому в дальнейшем удобней будет данные этих столбцов перевести в целочисленный формат.

Проверим данные на дубликаты.

In [5]:
# проверка данных на дубликаты
duplicate_count = df.duplicated().sum()
duplicate_count

153

In [6]:
# вывод на экран таблицы с дубликатами данных
print(df[df.duplicated()])

      Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
281     1     39.0   48100.0            1                  0
488     1     24.0   32900.0            1                  0
513     0     31.0   37400.0            2                  0
718     1     22.0   32600.0            1                  0
785     0     20.0   35800.0            0                  0
...   ...      ...       ...          ...                ...
4793    1     24.0   37800.0            0                  0
4902    1     35.0   38700.0            1                  0
4935    1     19.0   32700.0            0                  0
4945    1     21.0   45800.0            0                  0
4965    0     22.0   40100.0            1                  0

[153 rows x 5 columns]


In [7]:
# вывод на экран доли дубликатов в данных
print('Доля дубликатов в данных: {:.2%}'.format(duplicate_count/len(df)))

Доля дубликатов в данных: 3.06%


В данных 153 строки с дубликатами. Так как в данных всего пять столбцов, а Фамилий клиентов или их ID у нас нет, то возможно удаление строк с просто совпадающими данными . Конечно, вероятность из 5000 клиентов получить 153 строки с полностью совпадающими данными о зарплате, возрасте, поле клиента и количестве членов семьи, а также значениями целевого признака исчезающе мала. 3 % для подобных данных не похоже на случайность. Поэтому удалим эти строки.

In [8]:
# удаление дубликатов
df = df.drop_duplicates().reset_index(drop=True)

Выведем на экран максимальные, минимальные, средние и медианные значения по всем признакам, в том числе по целевому.

In [9]:
# вывод на экран максимальных, минимальных, средних и медианных значений признаков датасета
df.describe().loc[['min', 'mean', '50%', 'max']]

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
min,0.000000,18.000000,5300.000000,0.000000,0.000000
mean,0.498453,31.023932,39895.811842,1.203425,0.152259
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


В данных женщин и мужчин почти поровну. Возраст и зарплаты распределены нормально, аномалий не наблюдается. Данные с количеством членов семьи и количеством страховых выплат смещены к минимальным значениям, что логично ведь большинство не имеют детей или у них 1 или 2 ребенка. А большое количество страховых выплат должно быть довольно редким явлением.

Преобразум все данные к целочисленному типу, так как возраст и зарплата здесь не удобны для чтения в вещественном формате. 

In [10]:
# преобразование типа данных в целочисленный 
df = df.astype('int')
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0
...,...,...,...,...,...
4842,0,28,35700,2,0
4843,0,34,52400,1,0
4844,0,20,33900,2,0
4845,1,22,32700,3,0


## Шаг 2.  Проверка гипотезы о неизменности качества модели после трансформации признаков.

**Нулевая гипотеза:** При умножении матрицы признаков на обратимую матрицу - качество модели "Линейной регрессии" не изменится.

**Альтернативная гипотеза:** При умножении матрицы признаков на обратимую матрицу - качество модели "Линейной регрессии" изменится.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- $a$ — предсказания на исходной матрице признаков

- $b$ — gредсказания на преобразованной матрице признаков

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Если признаки умножают на обратимую матрицу, изменится ли качество "Линейной регрессии"?

**Ответ:** Качество "Линейной регрессии" при перемножении матрицы признаков на обратимую матрицу не изменится.


**Обоснование:** 
Исходная матрица признаков X(нулевой столбец состоит из единиц). 

    Предсказания на исходной матрице признаков:
$$ a = Xw $$
    Предсказания на преобразованной матрице признаков:
$$ b = XPw $$
    Формула обучения на преобразованной матрице признаков:
$$ w_b = (XP^T*XP)^{-1}*XP^T*y$$
    Подставим в формулу предсказания на преобразованной матрице признаков - формулу обучения на преобразованной матрице признаков:
$$ b = XP*((XP^T*XP)^{-1}*XP^T*y) $$
    Для решения задачи нам понадобятся формулы свойств перемножаемых матриц:
    
   1) Любая обратимая матрица, умноженная сама на себя равна единичной матрице:
$$P*P^{-1} = E$$
   2) Любая матрица, умноженная на единичную матрицу равна самой себе: 
$$ X*E = X $$
   3) Обратная матрица от перемножаемых матриц равна произвендению второй обратной матрицы на первую обратную матицу:           $$ (XP)^{-1} = P^{-1}*X^{-1} $$
   4) Транспонированная матрица от перемножаемых матриц равна произведению второй транспонированной матрицы на первую транспонированную матрицу:
$$ (XP)^T = P^T*X^T $$
    
Используя, вышеперечисленные правила упростим формулу предсказания на преобразованной матрице признаков в несколько этапов:

   1)$$ b = X_{5000\times 4}P_{4\times 4}*((X_{5000\times 4}P_{4\times 4})^{T}*(X_{5000\times 4}P_{4\times 4}))^{-1}*(X_{5000\times 4}P_{4\times 4})^T*y $$
   
   2)$$ b = (X_{5000\times 4}P_{4\times 4})*(P_{4\times 4}^T*(X_{4\times 5000}^T X_{5000\times 4})_{4\times 4}*P_{4\times 4})^{-1}* P_{4\times 4}^T*X_{4\times 5000}^T*y$$  
    
   3)$$ b = X_{5000\times 4}*(P_{4\times 4}* P_{4\times 4}^{-1})* (X_{4\times 5000}^T X_{5000\times 4})_{4\times 4})^{-1}* ((P_{4\times 4}^T)^{-1}*P_{4\times 4}^T)* X_{4\times 5000}^T*y$$ 
    
   4)$$ b = (X_{5000\times 4}*E_{4\times 4})* (X_{4\times 5000}^T X_{5000\times 4})_{4\times 4})^{-1}* (E_{4\times 4}* X_{4\times 5000}^T)*y$$ 
    
   5)$$ b =  X_{5000\times 4}*(X_{4\times 5000}^T X_{5000\times 4})_{4\times 4})^{-1}*X_{4\times 5000}^T*y$$ 
    
   6)$$ b =  X*(X^T*X)^{-1}*X^T*y$$  
    
Формула обучения "Линейной регрессии" для преобразованной матрицы признаков тождественна формуле с признаками до преобразования. **Принимаем нулевую гипотезу о неизменности качества модели "Линейной регресии" и делаем вывод, что предсказания на матрице признаков до преобразования и после перемножения матрицы признаков на обратимую матрицу идеинтичны:**

  $$ a = b $$



## Шаг 3. Алгоритм преобразования матрицы признаков.

**Алгоритм преобразования матрицы с признаками:** 

 - Разделение данных исходного датасета на признаки и целевой признак.
 - Преобразование данных признаков и целевого признака в массив.
 - Добавление столбца из единиц с индексом "0" в матрицу признаков. 
 - Генерация случайной матрицы, которая будет использоваться как ключ шифрования.
 - Создание обратной матрицы для случайной матрицы-шифрования.
 - Умножение матрицы признаков на ключ шифрования.
 - Проверка модели на исходных и преобразованных признаках.
 - Дешифрование преобразованной матрицы признаков при помощи обратной матрицы к обратимой матрице-шифрования.


**Обоснование для преобразования матрицы с признаками:**

- Количество признаков в преобразованной матрице должно быть равно количеству признаков в исходной матрице, поэтому размерность матрицы, используемой как ключ шифрования должно соотвестствовать числу признаков в данных.
- Матрица, используемая для шифрования должна иметь квадратную форму и иметь обратную, то есть являться невырожденной.
- Произведение строк данных исходной матрицы на столбцы обратимой невырожденной матрицы шифрования должно изменять равномерно веса каждого признака для всех строк, так как строки исходной матрицы умножаются на одни и те же столбцы квадратной матрицы.


## Шаг 4. Проверка алгоритма преобразования матрицы признаков.

### Шаг 4.1. Подготовительный этап реализации алгоритма преобразования.

In [11]:
# разделение данных на признаки и целевой признак
X, y = df.drop('Страховые выплаты', axis=1), df['Страховые выплаты']

In [12]:
# преобразование данных с признаками в массив
X = np.array(X)
X

array([[    1,    41, 49600,     1],
       [    0,    46, 38000,     1],
       [    0,    29, 21000,     0],
       ...,
       [    0,    20, 33900,     2],
       [    1,    22, 32700,     3],
       [    1,    28, 40600,     1]])

In [13]:
# присоединение нулевого столбца с единицами к массиву с признаками
X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
X

array([[1.00e+00, 1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
       [1.00e+00, 0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
       [1.00e+00, 0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
       ...,
       [1.00e+00, 0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
       [1.00e+00, 1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
       [1.00e+00, 1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]])

In [14]:
# создание обратимой случайной матрицы-ключа шифрования размерностью 5 Х 5
np.random.seed(42)
P = np.random.normal(size=(5,5))
P

array([[ 0.49671415, -0.1382643 ,  0.64768854,  1.52302986, -0.23415337],
       [-0.23413696,  1.57921282,  0.76743473, -0.46947439,  0.54256004],
       [-0.46341769, -0.46572975,  0.24196227, -1.91328024, -1.72491783],
       [-0.56228753, -1.01283112,  0.31424733, -0.90802408, -1.4123037 ],
       [ 1.46564877, -0.2257763 ,  0.0675282 , -1.42474819, -0.54438272]])

In [15]:
# создание обратной матрицы для случайной матрицы-ключа шифрования
P_inv = np.linalg.inv(P)
P_inv 

array([[ 0.27160427, -0.13802315,  0.1068986 , -0.40131296,  0.44803328],
       [ 0.43517254, -0.07793672,  1.47897661, -1.76590496, -0.36977934],
       [ 0.00734044,  0.90858841, -1.49597684,  2.00568236,  0.43912026],
       [ 0.49145763, -0.27331144,  0.46849253, -0.64008524, -0.30765249],
       [-0.734562  ,  0.48873361, -1.7372825 ,  1.57594284,  0.38231772]])

In [16]:
# получение единичной матрицы умножением обратимой случайной матрицы на ее обратную
E_p = np.around(np.dot(P,P_inv))
E_p

array([[ 1., -0.,  0., -0., -0.],
       [ 0.,  1., -0., -0.,  0.],
       [-0., -0.,  1.,  0.,  0.],
       [-0.,  0., -0.,  1.,  0.],
       [-0.,  0.,  0., -0.,  1.]])

In [17]:
# создание зашифрованной матрицы путем умножения исходной на случайную обратимую матрицу
X_crypt = np.dot(X, P)
X_crypt

array([[-27906.73334979, -50254.30331627,  15598.07080133,
        -45116.8098286 , -70121.22119342],
       [-21386.2809621 , -38509.37018197,  11953.24411986,
        -34592.82747939, -53747.66540714],
       [-11820.980513  , -21283.09795418,   6606.85857891,
        -19122.46768318, -29708.63449856],
       ...,
       [-19067.38758343, -34344.87939131,  10658.60656536,
        -30821.60823158, -47912.91675074],
       [-18392.33787192, -33129.0600699 ,  10282.82865372,
        -29737.70012401, -46221.60396748],
       [-22840.12115662, -41132.76874646,  12766.69929844,
        -36919.72050573, -57388.06394958]])

После умножения исходной матрицы с признаками на случайную обратимую заметно что цифры не имеет вид базы данных клиентов страховой компании. 

### Шаг 4.2. Проверка модели на признаках до трансформации.

In [18]:
# деление общей выборки с признаками до преобразования на тренировочную и тестовую 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12345)

In [19]:
# создание pipline модели "Линейная регрессия" с масштабированием признаков, ее обучение, получение предсказания и вывод на
# экран метрики R2_score
model = LinearRegression()
scaler = StandardScaler()
LR = Pipeline([('scale', scaler), ('Linear_regression', model)])
LR.fit(X_train, y_train)
predictions = LR.predict(X_test)
print(r2_score(y_test, predictions))

0.4230772761583671


### Шаг 4.3. Проверка модели на признаках после трансформации.

In [20]:
# деление общей выборки с признаками после преобразования на тренировочную и тестовую 
X_crypt_train, X_crypt_test, y_train, y_test = train_test_split(X_crypt, y, test_size=0.25, random_state=12345)

In [21]:
# обучение, получение предсказания модели "Линейной регрессии" на преобразованной выборке и вывод на экран метрики R2_score
LR.fit(X_crypt_train, y_train)
predictions = LR.predict(X_crypt_test)
print(r2_score(y_test, predictions))

0.42307727615835844


 - При проверке алгоритма выяснили, что качество модели "Линейной регрессии" не изменилось.
 - Метрика R2 (коэффициент детерминации) при тестировании модели на признаках без изменений и после умножения матрицы с признаками на случайную обратимую осталась прежней.

### Шаг 4.4. Дешифрование преобразованной матрицы признаков.

Далее применим умножение матрицы зашифрованных данных с признаками на обратную матрицу случайной матрице-шифрования, чтобы вернуть данные к исходному виду.

In [22]:
# создание датасета признаков с исходными данными путем перемножения зашифрованной матрицы на обратную матрице-шифрования
X_uncrypt = pd.DataFrame(data = X_crypt @ P_inv, columns = ['1', 'Пол', 'Возраст', 'Зарплата', 'Члены семьи'])
X_uncrypt

,1,Пол,Возраст,Зарплата,Члены семьи
0,1.0,1.000000e+00,41.0,49600.0,1.000000e+00
1,1.0,-5.224351e-13,46.0,38000.0,1.000000e+00
2,1.0,-2.825123e-12,29.0,21000.0,-1.238557e-12
3,1.0,2.806707e-12,21.0,41700.0,2.000000e+00
4,1.0,1.000000e+00,28.0,26100.0,2.066104e-12
...,...,...,...,...,...
4842,1.0,-4.123279e-12,28.0,35700.0,2.000000e+00
4843,1.0,4.557182e-12,34.0,52400.0,1.000000e+00
4844,1.0,-5.552812e-13,20.0,33900.0,2.000000e+00
4845,1.0,1.000000e+00,22.0,32700.0,3.000000e+00


In [23]:
# удаление нулевого столбца с единицами
X_uncrypt = X_uncrypt.drop(columns='1', axis=1).reset_index(drop=True)
X_uncrypt

,Пол,Возраст,Зарплата,Члены семьи
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,-5.224351e-13,46.0,38000.0,1.000000e+00
2,-2.825123e-12,29.0,21000.0,-1.238557e-12
3,2.806707e-12,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,2.066104e-12
...,...,...,...,...
4842,-4.123279e-12,28.0,35700.0,2.000000e+00
4843,4.557182e-12,34.0,52400.0,1.000000e+00
4844,-5.552812e-13,20.0,33900.0,2.000000e+00
4845,1.000000e+00,22.0,32700.0,3.000000e+00


In [24]:
# равномерное (банковское) округление данных
X_uncrypt = np.around(X_uncrypt).astype('int')
X_uncrypt

,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0
...,...,...,...,...
4842,0,28,35700,2
4843,0,34,52400,1
4844,0,20,33900,2
4845,1,22,32700,3


In [25]:
# объединение дешифрованного датасета признаков с данными целевого признака
df_uncrypt = X_uncrypt.join(y)
df_uncrypt

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0
...,...,...,...,...,...
4842,0,28,35700,2,0
4843,0,34,52400,1,0
4844,0,20,33900,2,0
4845,1,22,32700,3,0


 - После реализации этапа дешифрования данных, путем умножения зашифрованной матрицы на обратную матрице-шифрования, а также соединения полученных данных с целевым признаком, получили исходный датасет.
 - Таким образом, умножая матрицу признаков на случайную обратимую матрицу, возможно защитить данные клиентов страховой компании.


## Шаг 5. Итоги исследования.

### Шаг 5.1. Описание хода исследования.

Для реализации поставленной задачи мы:
   - Загрузили данные.
   - Проверили датасет на дубликаты, удалили их.
   - Изучили максимальные, минимальные, средние и медианные значения признаков..
   - Привели данные в целочисленный формат.
   - Проверили гипотезу о неизменности качества модели "Линейной регрессии" после перемножения матрицы признаков с обратимой случайной матрицей, подтвердили её.
   - Написали алгоритм преобразования матрицы признаков.
   - Создали зашифрованную матрицу.
   - Проверили качество модели, рассчитав коэффициент детерминации "Линейной регрессии" на исходной матрице признаков и преобразованной.
   - Произвели дешифрование матрицы с признаками и вернули датасет к исходному виду.

По итогам исследования делаем общий вывод:

### Шаг 5.2. Общий вывод.

- При умножении матрицы признаков на обратимую матрицу - качество модели "Линейной регрессии" не изменится.
- При проверке алгоритма выяснили, что качество модели "Линейной регрессии" не изменилось, коэффициент детерминации один для модели с исходными признаками и зашифрованными.
- Данные возвращаются к исходному виду путём перемножения зашифрованной матрицы признаков на обратную к случайной матрице-шифрования.  